## Visualizing optimizers
This notebook visualizes common optimizers used during neural networks training. The idea is based on these visualizations: https://imgur.com/a/Hqolp#NKsFHJb

The goal was to reproduce similar animations and try different surfaces.

Unfortunately matplotlib is not suitable for 3D plotting of several elements on the same plot (surface and lines for optimizers): it can't correctly infer overlapping since the surface is treated as 2D image once it is created.

So I searched for alternatives and decided to use PyVista, which supports true 3D rendering.

Generating GIFs from PyVista turned out to be problematic: lines and legend kept changing colors between frames so I had to use some workarounds.

## Preparation part: methods and initialization

In [5]:
import numpy as np
import tensorflow as tf
import keras
import pyvista as pv
from pyvista import examples
from matplotlib import colors

2022-11-22 13:26:12.086892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


The next cell is the workaround to produce consistent frames for GIF with PyVista: reduce the amount of colors. Since standard matplotlib's color maps have lots of color shades, I decided to have simpler color map - just 11 colors in it moving from red to blue.

In [6]:
surface_cmap = colors.ListedColormap([(1.0-i/20, 0.5, 0.5+i/20) for i in range(0, 11)])

In [7]:
def evaluate_on_grid(scale, func):
    """Creates XY grid and evaluates function on it to produce Z.

    Parameters:
    scale -- defines range for X and Y to be (-scale, scale)
    func -- function to evaluate on XY, must take two parameters: x, y. Must produce tf.Tensor.
    """
    x = np.arange(-scale, scale, scale / 20, dtype='float32')
    y = np.arange(-scale, scale, scale / 20, dtype='float32')
    x, y = np.meshgrid(x, y)
    z = func(x, y).numpy()
    return x, y, z

In [8]:
def show_surface(x, y, z):
    """Renders surface based on XYZ grid."""
    grid = pv.StructuredGrid(x, y, z)
    grid['scalars'] = -z.ravel(order='f')
    plotter = pv.Plotter() 
    plotter.add_mesh(grid, scalars='scalars', cmap=surface_cmap, show_edges=True, opacity=1.0)
    plotter.show()

In [9]:
def lines_optimizer(opt, func, start_x, start_y, vertical_adjustment, number_of_steps=100):
    """Calculates optimizer line which represents the path of gradient descent for the specified function.

    Parameters:
    opt -- instance of optimizer from `keras.optimizers` package.
    func -- function we want to minimize with the optimizer
    start_x, start_y -- starting point for optimizer
    vertical_adjustment -- technical attribute, defines the elevation of optimizer line to avoid intersecting optimizer lines with surface and between each other, should me some small value
    number_of_steps -- number of steps for optimizer to make
    
    Returns:
    list of 3-elements lists, which define line segments in 3D
    """
    varx = tf.Variable(start_x)
    vary = tf.Variable(start_y)
    loss = lambda: func(varx, vary)
    result = [[varx.numpy(), vary.numpy(), func(varx.numpy(), vary.numpy())+vertical_adjustment]]
    stopped = False
    for i in range(0, number_of_steps):
        if not stopped:
            step_count = opt.minimize(loss, [varx, vary]).numpy()
        func_value = func(varx.numpy(), vary.numpy())
        if abs(func_value) > 1000:
            stopped = True
        result.append([varx.numpy(), vary.numpy(), func_value+vertical_adjustment])
    return result

In [20]:
def create_animation(x, y, z, optimizer_lines, filename):
    """Creates GIF animation and saves it to the specified file.

    Parameters:
    x, y, z -- grid to render surface
    optimizer_lines -- list of 3-element lists, which define line segments in 3D
    filename -- file to save GIF to
    """
    grid = pv.StructuredGrid(x, y, z)
    grid['scalars'] = -z.ravel(order='f')

    plotter = pv.Plotter(off_screen=True, lighting=None) 
    #plotter.open_movie("anim.mp4")
    plotter.open_gif(filename)
    
    plotter.add_legend(legend, bcolor=(0.1, 0.1, 0.1))

    plotter.add_mesh(grid, scalars='scalars', cmap=surface_cmap, show_edges=True, opacity=1.0, show_scalar_bar=False)

    actors = []
    for i in range(2, gradient_steps):
        for actor in actors:
            plotter.remove_actor(actor)
        actors = []
        for opt_ind in range(0, len(optimizers)):
            actors.append(plotter.add_mesh(pv.MultipleLines(optimizer_lines[opt_ind][:i]), scalars=[0]*i, cmap=cmaps[opt_ind], line_width=4, show_scalar_bar=False))
        plotter.write_frame()

    plotter.close()

## List of optimizers to test
Here is the list of optimizers we will test, feel free to modify the list.
Note that during the simulation SGD and SGD+momentum were so slow that I decided to give them unfair advantage - increasing their learning rate, just to get more action out of them.

In [11]:
#list of tuples (label, color, optimizer)
#SGD and SGD+momentum were so slow during simulations that their learning rate was increased to have better visualization
optimizers = [('SGD', (1.0, 0.1, 0.1), keras.optimizers.SGD(learning_rate=0.05)),
              ('SGD+momentum', (1.0, 0.8, 0.1), keras.optimizers.SGD(learning_rate=0.05, momentum=0.5)),
              ('RMSprop', (0.0, 1.0, 0.1), keras.optimizers.RMSprop(learning_rate=0.01)),
              ('Adam', (0.0, 0.5, 1.0), keras.optimizers.Adam(learning_rate=0.01))
             ]

cmaps = [colors.ListedColormap([opt[1]]) for opt in optimizers]

#precalculate legend for the plot
legend = [[opt[0], opt[1]] for opt in optimizers]

## Fun part: let's try optimizers in action

### Simple slope
The simplest example is just a sloped plane. It has decent slope in the direction of Y and a tiny slope in the direction of X. Classic gradient descent optimizers respond to this accordingly: they go alongside Y and almost ignore X direction. But note how RMSprop and Adam treat both slopes equally! Don't be confused by SGDs beating RMSprop and Adam by such a large margin - remember that SGDs has 5x times larger learning rate in our simulation.

In [12]:
def f(x, y):
    return tf.identity(-y-0.001*x)

x, y, z = evaluate_on_grid(0.5, f)
#show_surface(x, y, z)

start_x = np.float64(0.01)
start_y = np.min(y) * 0.9

filename = 'simple-slope.gif'

gradient_steps = 100

vertical_adjustment = (np.max(z) - np.min(z))/30
optimizer_lines = [lines_optimizer(opt[2], f, start_x, start_y, (ind+1)*vertical_adjustment, number_of_steps=gradient_steps) for ind, opt in enumerate(optimizers)]

create_animation(x, y, z, optimizer_lines, filename)

2022-11-22 13:26:53.887256: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


![animation](simple-slope.gif "Simple slope")

### Parabolic saddle
Now the classic shape with a saddle point - parabolic shape: y^2 - x^2. The optimizers start at x close to 0, which makes the derivative over X axis close to 0 as well. Note how classic gradient descent optimizers struggle with this small derivative. RMSprop and Adam on the other hand have absolutely no problems.

In [13]:
def f(x, y):
    return tf.square(y) - tf.square(x)

filename = 'parabolic-saddle.gif'

x, y, z = evaluate_on_grid(0.5, f)
#show_surface(x, y, z)

start_x = np.float64(0.00001)
start_y = np.min(y) * 0.9

gradient_steps = 100

vertical_adjustment = (np.max(z) - np.min(z))/30
optimizer_lines = [lines_optimizer(opt[2], f, start_x, start_y, (ind+1)*vertical_adjustment, number_of_steps=gradient_steps) for ind, opt in enumerate(optimizers)]

create_animation(x, y, z, optimizer_lines, filename)


![animation](parabolic-saddle.gif "Parabolic saddle")

### Sloped cylinder
Well, technically not a cylinder but a parabola, but it doesn't matter here. The parabola is extended along X axis with a small slope. RMSprop and Adam again respect even small slope, nothing new here. But note how quick Adam is!

(Below is my attempt to explain these differences, but take it with some grain of salt, since I haven't invested much into analysis) 

RMSprop and SGD+momentum have similar speed, but most likely due to different reasons: RMSprop due to rescaling of gradient, SGD+momentum due to momentum. Adam has both of these and additionally has second-order momentum.

In [14]:
def f(x, y):
    return tf.square(y) - 0.1*x

x, y, z = evaluate_on_grid(0.5, f)
#show_surface(x, y, z)

start_x = np.min(x) * 0.9
start_y = np.min(y) * 0.9

filename = 'sloped-cylinder.gif'

gradient_steps = 100

vertical_adjustment = (np.max(z) - np.min(z))/30
optimizer_lines = [lines_optimizer(opt[2], f, start_x, start_y, (ind+1)*vertical_adjustment, number_of_steps=gradient_steps) for ind, opt in enumerate(optimizers)]

create_animation(x, y, z, optimizer_lines, filename)

![animation](sloped-cylinder.gif "Sloped cylinder")

### Going around hill
A bit more complex version of saddle point just to have some fun. See how presence of momentum brings success here: both SGD+momentum and Adam reached the bottom quickly (well, for SGD+momentum sort of, since it has 5x learning rate). RMSprop is the last one but remember it has 5x lower rate than SGDs.

Also note how Adam does some exploration before converging to the minimum.

In [15]:
def f(x, y):
    return tf.sin(3+ 1/(tf.square(x) + 0.45)) + tf.sin(3 + 1/(tf.square(y) + 0.45)) - 0.6*y

x, y, z = evaluate_on_grid(0.5, f)
#show_surface(x, y, z)

start_x = np.float64(0.01)
start_y = np.min(y) * 0.9

filename = 'around-hill.gif'

gradient_steps = 100

vertical_adjustment = (np.max(z) - np.min(z))/30
optimizer_lines = [lines_optimizer(opt[2], f, start_x, start_y, (ind+1)*vertical_adjustment, number_of_steps=gradient_steps) for ind, opt in enumerate(optimizers)]

create_animation(x, y, z, optimizer_lines, filename)

![animation](around-hill.gif "Around hill")

### Going around hill (with local minimum)
This one was unexpected. When playing with previous example I've suddenly discovered case which demonstrates Adam's ability to overcome local minimum.

In [16]:
def f(x, y):
    return tf.sin(3+ 1/(tf.square(x) + 0.45)) + tf.sin(3 + 1/(tf.square(y) + 0.45)) - 0.5*y

x, y, z = evaluate_on_grid(0.5, f)
#show_surface(x, y, z)

start_x = np.float64(0.01)
start_y = np.min(y) * 0.9

filename = 'around-hill2.gif'

gradient_steps = 100

vertical_adjustment = (np.max(z) - np.min(z))/30
optimizer_lines = [lines_optimizer(opt[2], f, start_x, start_y, (ind+1)*vertical_adjustment, number_of_steps=gradient_steps) for ind, opt in enumerate(optimizers)]

create_animation(x, y, z, optimizer_lines, filename)

![animation](around-hill2.gif "Around hill 2")

### Sloped trenches
Trenches have slope in the Y direction and have local minima. There is also a tiny slope in the X direction. Another example when Adam overcomes local minima. Both RMSprop and Adam respond to tiny slope as before. SGDs are paralyzed.

In [18]:
def f(x, y):
    return  tf.square(tf.sin(20*y+3)/10+0.1)- 0.3*y - 0.0001*x

x, y, z = evaluate_on_grid(0.5, f)
#show_surface(x, y, z)

start_x = np.min(x) * 0.9
start_y = np.min(y) * 0.8

filename = 'sloped-trenches.gif'

gradient_steps = 100

vertical_adjustment = (np.max(z) - np.min(z))/30
optimizer_lines = [lines_optimizer(opt[2], f, start_x, start_y, (ind+1)*vertical_adjustment, number_of_steps=gradient_steps) for ind, opt in enumerate(optimizers)]

create_animation(x, y, z, optimizer_lines, filename)

![animation](sloped-trenches.gif "Sloped trenches")

### Circular pit
Another surface causing troubles for SGDs and being overcomed by Adam easily. Note oscillation of Adam due to its momentum.

In [19]:
def f(x, y):
    return  tf.sin(tf.sqrt(tf.square(15*x) + tf.square(15*y)))/10 - 0.1*y

x, y, z = evaluate_on_grid(0.5, f)
#show_surface(x, y, z)

start_x = np.float64(0.01)
start_y = np.min(y) * 0.9

filename = 'circular-pit.gif'

gradient_steps = 100

vertical_adjustment = (np.max(z) - np.min(z))/30
optimizer_lines = [lines_optimizer(opt[2], f, start_x, start_y, (ind+1)*vertical_adjustment, number_of_steps=gradient_steps) for ind, opt in enumerate(optimizers)]

create_animation(x, y, z, optimizer_lines, filename)

![animation](circular-pit.gif "Circular pit")